In [ ]:
import pandas as pd
import glob
import os
import ephys_analysis.funcs_for_results_tables as get_results
import ephys_analysis.funcs_plot_intrinsic_props as pl_intr
import ephys_analysis.funcs_sorting as sort
import json
import numpy as np

# Collect dfs. Slice - treatment pairs

In [ ]:
SAVE_DIR = '/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/treatment_cross_check/'

# intrinsic df to OP241120
df_intr_complete = pd.read_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/summary_data_tables/intrinsic_properties/2024-11-27_collected.xlsx')
# df_intr = get_results.collect_intrinsic_df() # for most unpdated version

# adding RMPs from charact
# df_intr_complete = pl_intr.get_column_RMPs_from_char(df_intr)

# only slice info 
df_intr = df_intr_complete[['OP', 'patcher', 'slice', 'treatment']]
minis_df = get_results.collect_events_dfs('minis')[['OP', 'slice', 'patcher', 'treatment']]
spontan_df = get_results.collect_events_dfs('spontan')[['OP', 'slice', 'patcher', 'treatment']]

# merge all dfs
collected_dfs = pd.concat([df_intr, minis_df, spontan_df], axis=0).drop_duplicates()

In [ ]:
HUMAN_DIR = '/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/data/human/'
PATHS = [HUMAN_DIR + 'data_verji/', HUMAN_DIR + 'data_rosie/']

# collect from json meta_active_chans 
patchers_json, OPs_json, slices_json, treatments_json = [], [], [], []
for dir in PATHS:
    op_dirs = [d for d in os.listdir(dir) if d.startswith('OP')]
    if 'rosie' in dir:
        patcher = ['Rosie']
    else:
        patcher = ['Verji']
    for op_dir in op_dirs:
        # meta active chans
        if op_dir == 'OP201020':
            continue
        meta_active_chans = glob.glob(f"{dir + op_dir}/*meta_active_chans.json")
        if len(meta_active_chans) < 1:
            # print('no meta_active_chans for ' + op_dir)
            continue
        elif len(meta_active_chans) >= 4:
            active_chans_json = json.load(open(meta_active_chans[0]))
            slices_json = slices_json + list(active_chans_json[3].keys())
            treatments_json = treatments_json + list(active_chans_json[3].values())
            patchers_json = patchers_json + patcher * len(active_chans_json[3].keys())
            OPs_json = OPs_json + [op_dir] * len(active_chans_json[3].keys())
            continue
        elif len(meta_active_chans) < 4:
            active_chans_json = json.load(open(meta_active_chans[0]))
            slices_json = slices_json + active_chans_json[0]['slices']
            treatments_json = treatments_json + active_chans_json[0]['treatment']
            slices_json =  slices_json + active_chans_json[2]['mini_slices']
            treatments_json = treatments_json + active_chans_json[2]['treatment']
            OPs_json = OPs_json + [op_dir] * (len(active_chans_json[0]['slices']) \
                                                        + len(active_chans_json[2]['mini_slices']))
            patchers_json = patchers_json + patcher * (len(active_chans_json[0]['slices']) \
                                                        + len(active_chans_json[2]['mini_slices']))

# fixing the treatment, when list
tr = []
L = treatments_json
for x in L:
    if isinstance(x, list):
        if len(x) < 1:
            x = 'no treatment'
        else:
            x = x[0]
    tr.append(x)
treatments_json = tr

# creating df from json meta_active_chans
df_from_json = pd.DataFrame({'patcher': patchers_json, 'OP': OPs_json, 'slice': slices_json, 'treatment': treatments_json})
df_from_json = df_from_json.drop_duplicates()

In [ ]:
# connectivity slices

OPs_con, patchers_con, slices_con, treatments_con = [], [], [], []
for dir in PATHS:
    op_dirs = [d for d in os.listdir(dir) if d.startswith('OP')]
    if 'rosie' in dir:
        patcher = ['Rosie']
    else:
        patcher = ['Verji']
    for op_dir in op_dirs:
        con_screen_jsons = glob.glob(f"{dir + op_dir}/*con_screen_only.json")
        if len(con_screen_jsons) < 1:
            # print('no con_screen_json for ' + op_dir)
            continue
        else:
            con_screen_json = json.load(open(con_screen_jsons[0]))[0]
            OPs_con = OPs_con + [op_dir] * len(con_screen_json['slices'])
            patchers_con = patchers_con + patcher * len(con_screen_json['slices'])
            slices_con = slices_con + con_screen_json['slices']
            treatments_con = treatments_con + con_screen_json['treatment']

df_con_jsons = pd.DataFrame({'patcher': patchers_con, 'OP': OPs_con, 'slice': slices_con, \
                             'treatment': treatments_con}).drop_duplicates()

In [ ]:
all_df = pd.concat([collected_dfs, df_from_json, df_con_jsons], axis=0).drop_duplicates()

## Compare the slice treatment pairs

In [ ]:
all_df = all_df.sort_values(by=['patcher', 'OP', 'slice'])
all_df.to_excel(SAVE_DIR + 'all_df.xlsx', index=False)

# Reanalysis

In [1]:
import os
import pandas as pd
import glob
import ephys_analysis.funcs_sorting as sort
import ephys_analysis.funcs_for_results_tables as get_results
import ephys_analysis.funcs_human_characterisation as hcf
import json

human_dir = '/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/data/human/'

OP = 'OP210319'
patcher = 'Rosie'
tissue_source = 'Virchow'
age = 9

/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/code/Human-slice-scripts/src/ephys_analysis/detect_peaks.py:11: UserWarning: A newest version is available at https://pypi.org/project/detecta/
  warnings.warn('A newest version is available at https://pypi.org/project/detecta/')


In [2]:
work_dir, filenames, indices_dict, slice_names = sort.get_OP_metadata(human_dir, OP, patcher, 'old')

# add correct vc_end to indices_dict
if len(indices_dict['vc']) != len(indices_dict['vc_end']):
    vc_ends = []
    for vc in indices_dict['vc']:
        vc_ends.append(input('vc_end_fn ' + str(vc))) # 000 when no vc_end

    if len(indices_dict['vc']) == len(vc_ends):
        indices_dict['vc_end'] = vc_ends

    # save the fixed json
    save_json_path = work_dir + OP + '_indices_dict.json'
    with open(save_json_path, 'w') as file:
        json.dump(indices_dict, file)

In [3]:
active_chans_meta = sort.get_json_meta(human_dir, OP, patcher, '_meta_active_chans.json')[0]  
cortex_out_time = sort.get_datetime_from_input(active_chans_meta['OP_time'][0])

In [ ]:
# 'filename' - characterizaiton 1 filename

df_OP = pd.DataFrame(columns=['fn_vc', 'filename', 'slice', 'cell_ch', 'cell_ID', 'day', 'treatment', 
    'hrs_incubation', 'repatch', 'hrs_after_OP', 'Rs', 'Rin', 'resting_potential', 'max_spikes', 
    'Rheobase', 'AP_heigth', 'TH', 'max_depol', 'max_repol', 'membra_time_constant_tau', 'capacitance', 
    'Rheobse_ramp', 'AP_halfwidth', 'RMP_from_char'])

In [ ]:
for i, indx_vc in enumerate(indices_dict['vc']):
    vc_end = indices_dict['vc_end'][i]
    active_chans = active_chans_meta['active_chans'][i]
    treatment = active_chans_meta['treatment'][i]
    slice_name = active_chans_meta['slices'][i]
    day = 'D1'
    if slice_name[-2:] == 'D2':
        day = 'D2'

    fn_vc = work_dir + filenames[indx_vc]

    time_after_op = sort.get_time_after_OP(fn_vc, cortex_out_time)
    cell_IDs = hcf.get_cell_IDs(fn_vc, slice_name, active_chans)

    Rs, Rin = hcf.get_access_resistance(fn_vc, active_chans)

    params1_df = pd.DataFrame({'fn_vc': filenames[indx_vc], 'slice' : slice_name, \
                                   'cell_ch': active_chans,'cell_ID':cell_IDs, 'day' : day , \
                                    'treatment': treatment, 'Rs' : Rs, 'Rin': Rin, }).reset_index(drop=True)
    
    tissue = pd.Series(tissue_source).repeat(len(params1_df))
    OPs = pd.Series(OP).repeat(len(params1_df))
    researcher = pd.Series(patcher).repeat(len(params1_df))
    patient_age = pd.Series(age).repeat(len(params1_df))
    series_df = pd.DataFrame({'tissue_source': tissue, 'OP': OPs, 'patcher': researcher, 'patient_age': patient_age}).reset_index(drop=True)

    df_vc = pd.concat([series_df, params1_df], axis = 1)
    df_OP = pd.concat([df_OP.loc[:], df_vc], axis = 0)
df_OP.to_excel(work_dir + 'data_tables/' + OP + '_Intrinsic_and_synaptic_properties_V1.xlsx', index=False) 


# get VC QC table
get_results.get_QC_access_resistance_df (human_dir, OP, patcher)

In [4]:
# build analysis pairs
dict_vc_char, dict_vc_spontan = {}, {}

for i, indx_vc in enumerate(indices_dict['vc']):
    key = indx_vc
    if i < len(indices_dict['vc']) - 1: # all but the last i
        next_key = indices_dict['vc'][i + 1]
        vals_char = [x for x in indices_dict['characterization'] if key < x < next_key]
        vals_spontan = [x for x in indices_dict['spontan'] if key < x < next_key]
    else: # last i
        vals_char = [x for x in indices_dict['characterization'] if key < x]
        vals_spontan = [x for x in indices_dict['spontan'] if key < x]
    dict_vc_char[key] = vals_char
    dict_vc_spontan[key] = vals_spontan


In [ ]:
df_char_all = pd.DataFrame(columns = ['max_spikes', 'Rheobase', 'AP_heigth', 'TH', 'max_depol', 'max_repol',
       'membra_time_constant_tau', 'capacitance', 'AP_halfwidth'])

fn_skip = []

for i, indx_vc in enumerate(dict_vc_char.keys()):

    for char_file in dict_vc_char[indx_vc]:

        fn_char = work_dir + filenames[char_file]
        active_channels = active_chans_meta['active_chans'][i]
        inj = hcf.get_inj_current_steps(fn_char)

        charact_params  = hcf.all_chracterization_params(fn_char, active_channels, inj)
        df_char = pd.DataFrame.from_dict(charact_params)
        RMPs = hcf.get_RMP_char_file(fn_char, active_channels)
        time_after_op = sort.get_time_after_OP(fn_char, cortex_out_time)

        df_char.insert(0, 'fn_char', filenames[char_file])
        df_char.insert(1, 'fn_char_num', filenames[char_file])
        df_char.insert(2, 'chans', active_channels)
        df_char.insert(3, 'inj', str([float(x) for x in inj]))
        df_char.insert(4, 'R MPs', RMPs)
        df_char.insert(5, 'hrs_after_OP', time_after_op)

        df_char_all = pd.concat([df_char_all.loc[:], df_char], axis = 0)


/var/folders/5y/f01m99kn5tx3chk04wp6sc440000gn/T/ipykernel_17424/4156013811.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_char_all = pd.concat([df_char_all.loc[:], df_char], axis = 0)


no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj
no 0mV step in the inj


In [7]:
inj

[-40.0, -80.0, -120.0, -160.0, -200.0]

In [ ]:
import ephys_analysis.funcs_plotting_raw_traces as pl_raw
